# Module 9.2 – Monitoring & Maintenance Interactive Demo

This notebook demonstrates model monitoring and maintenance techniques for semiconductor manufacturing, including:

- 🔬 **MLflow experiment tracking** with parameters, metrics, and artifacts
- 📊 **Drift detection** using PSI, KS-test, and Wasserstein distance
- 🚨 **Alert systems** with manufacturing-specific thresholds
- 📈 **Performance monitoring** with PWS and Estimated Loss metrics
- 🎯 **Interactive visualizations** for drift analysis

## Learning Objectives
By the end of this notebook, you will understand:
1. How to set up comprehensive model monitoring
2. Different types of drift and how to detect them
3. Manufacturing-specific metrics and their interpretation
4. How to create effective alert systems
5. Best practices for maintaining models in production

In [ ]:
# Required imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Import our monitoring pipeline
from pathlib import Path
import sys
sys.path.append(str(Path.cwd()))
exec(open('9.2-monitoring-maintenance-pipeline.py').read())

print("✅ Imports successful!")
print(f"📍 Working directory: {Path.cwd()}")

## 1. MLflow Setup and Basic Tracking

Let's start by setting up MLflow for experiment tracking. MLflow helps us keep track of different model versions, their parameters, and performance metrics.

In [ ]:
# Set up MLflow (if available)
if HAS_MLFLOW:
    import mlflow
    import mlflow.sklearn
    
    # Set experiment
    mlflow.set_experiment("semiconductor_monitoring_demo")
    print("✅ MLflow tracking enabled")
    print(f"📊 Experiment: {mlflow.get_experiment_by_name('semiconductor_monitoring_demo').experiment_id}")
else:
    print("⚠️  MLflow not available - using local tracking only")

# Create monitoring pipeline
pipeline = MonitoringPipeline()
if HAS_MLFLOW:
    pipeline.enable_mlflow("semiconductor_monitoring_demo")

print("🔧 Monitoring pipeline created")

## 2. Generate Synthetic Semiconductor Data

We'll create synthetic semiconductor process data that mimics real-world yield prediction scenarios.

In [ ]:
# Generate baseline training data
np.random.seed(42)
baseline_data = generate_yield_process(n=1000, seed=42)

print(f"📊 Generated dataset shape: {baseline_data.shape}")
print(f"📋 Features: {list(baseline_data.columns)}")
print(f"🎯 Target range: {baseline_data['target'].min():.2f} - {baseline_data['target'].max():.2f}")

# Display basic statistics
baseline_data.describe()

In [ ]:
# Visualize the baseline data
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Baseline Semiconductor Process Data', fontsize=16, fontweight='bold')

# Plot main process parameters
process_params = ['temperature', 'pressure', 'flow', 'time', 'target']
colors = ['red', 'blue', 'green', 'orange', 'purple']

for i, (param, color) in enumerate(zip(process_params, colors)):
    ax = axes[i//3, i%3]
    ax.hist(baseline_data[param], bins=30, alpha=0.7, color=color, edgecolor='black')
    ax.set_title(f'{param.title()} Distribution')
    ax.set_xlabel(param.title())
    ax.set_ylabel('Frequency')
    ax.grid(True, alpha=0.3)

# Correlation plot
ax = axes[1, 2]
correlation_matrix = baseline_data[process_params].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=ax)
ax.set_title('Feature Correlations')

plt.tight_layout()
plt.show()

print("📈 Baseline data visualization complete")

## 3. Train Baseline Model with Monitoring

Let's train a baseline model and establish our monitoring framework.

In [ ]:
# Prepare training data
feature_cols = [col for col in baseline_data.columns if col != 'target']
X_baseline = baseline_data[feature_cols]
y_baseline = baseline_data['target'].values

# Split for training and validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_baseline, y_baseline, test_size=0.2, random_state=42
)

print(f"📊 Training set: {X_train.shape}")
print(f"📊 Validation set: {X_val.shape}")

# Train the model
print("\n🔄 Training baseline model...")
pipeline.fit(X_train, y_train, model_type="ridge", alpha=1.0)

# Evaluate baseline performance
baseline_metrics = pipeline.evaluate(X_val, y_val)
print("\n✅ Baseline model trained successfully!")
print(f"📈 Baseline MAE: {baseline_metrics['mae']:.3f}")
print(f"📈 Baseline RMSE: {baseline_metrics['rmse']:.3f}")
print(f"📈 Baseline R²: {baseline_metrics['r2']:.3f}")
print(f"🎯 Prediction Within Spec: {baseline_metrics['pws_percent']:.1f}%")
print(f"💰 Estimated Loss: ${baseline_metrics['estimated_loss']:,.0f}")

## 4. Visualize Model Performance

Let's visualize how well our baseline model performs.

In [ ]:
# Make predictions for visualization
y_pred_val = pipeline.predict(X_val)

# Create performance visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Baseline Model Performance Analysis', fontsize=16, fontweight='bold')

# 1. Predicted vs Actual
ax1 = axes[0, 0]
ax1.scatter(y_val, y_pred_val, alpha=0.6, color='blue')
min_val, max_val = min(y_val.min(), y_pred_val.min()), max(y_val.max(), y_pred_val.max())
ax1.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
ax1.set_xlabel('Actual Yield')
ax1.set_ylabel('Predicted Yield')
ax1.set_title('Predicted vs Actual Values')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Residuals plot
ax2 = axes[0, 1]
residuals = y_pred_val - y_val
ax2.scatter(y_pred_val, residuals, alpha=0.6, color='green')
ax2.axhline(y=0, color='r', linestyle='--', lw=2)
ax2.set_xlabel('Predicted Values')
ax2.set_ylabel('Residuals')
ax2.set_title('Residuals Plot')
ax2.grid(True, alpha=0.3)

# 3. Error distribution
ax3 = axes[1, 0]
ax3.hist(residuals, bins=30, alpha=0.7, color='orange', edgecolor='black')
ax3.axvline(x=0, color='r', linestyle='--', lw=2)
ax3.set_xlabel('Prediction Error')
ax3.set_ylabel('Frequency')
ax3.set_title('Error Distribution')
ax3.grid(True, alpha=0.3)

# 4. Performance metrics summary
ax4 = axes[1, 1]
metrics_names = ['MAE', 'RMSE', 'R²', 'PWS %', 'Est. Loss ($K)']
metrics_values = [
    baseline_metrics['mae'],
    baseline_metrics['rmse'],
    baseline_metrics['r2'],
    baseline_metrics['pws_percent'],
    baseline_metrics['estimated_loss'] / 1000
]

bars = ax4.bar(metrics_names, metrics_values, color=['red', 'blue', 'green', 'orange', 'purple'], alpha=0.7)
ax4.set_title('Performance Metrics Summary')
ax4.set_ylabel('Value')
ax4.tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar, value in zip(bars, metrics_values):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{value:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("📊 Baseline model performance analysis complete")

## 5. Simulate Data Drift

Now let's simulate various types of drift that can occur in semiconductor manufacturing and see how our monitoring system detects them.

In [ ]:
# Generate different types of drifted data
print("🔄 Generating drift scenarios...")

# Scenario 1: Minor drift (equipment aging)
minor_drift_data = generate_drift_data(baseline_data, drift_strength=0.2)

# Scenario 2: Major drift (process recipe change)
major_drift_data = generate_drift_data(baseline_data, drift_strength=0.8)

# Scenario 3: Sudden shift (equipment maintenance)
sudden_shift_data = baseline_data.copy()
sudden_shift_data['temperature'] += 10  # 10°C temperature increase
sudden_shift_data['pressure'] += 0.2    # 0.2 Torr pressure increase

# Recalculate engineered features for sudden shift
sudden_shift_data['temp_centered'] = sudden_shift_data['temperature'] - sudden_shift_data['temperature'].mean()
sudden_shift_data['pressure_sq'] = sudden_shift_data['pressure'] ** 2
sudden_shift_data['temp_flow_inter'] = sudden_shift_data['temperature'] * sudden_shift_data['flow']

print("✅ Drift scenarios generated:")
print("   📉 Minor drift (equipment aging)")
print("   📉 Major drift (recipe change)")
print("   📉 Sudden shift (maintenance event)")

# Display drift characteristics
scenarios = {
    'Baseline': baseline_data,
    'Minor Drift': minor_drift_data,
    'Major Drift': major_drift_data,
    'Sudden Shift': sudden_shift_data
}

drift_summary = pd.DataFrame({
    scenario: {
        'Temp Mean': data['temperature'].mean(),
        'Temp Std': data['temperature'].std(),
        'Pressure Mean': data['pressure'].mean(),
        'Pressure Std': data['pressure'].std()
    }
    for scenario, data in scenarios.items()
}).T

print("\n📊 Drift Summary:")
print(drift_summary.round(3))

## 6. Visualize Drift Patterns

Let's visualize how different types of drift affect our process parameters.

In [ ]:
# Create comprehensive drift visualization
fig, axes = plt.subplots(3, 2, figsize=(15, 18))
fig.suptitle('Drift Detection Analysis: Process Parameter Changes', fontsize=16, fontweight='bold')

parameters = ['temperature', 'pressure']
colors = ['red', 'orange', 'darkred', 'blue']
scenario_names = ['Baseline', 'Minor Drift', 'Major Drift', 'Sudden Shift']

for i, param in enumerate(parameters):
    # Distribution comparison
    ax_dist = axes[i, 0]
    for j, (scenario, data) in enumerate(scenarios.items()):
        ax_dist.hist(data[param], bins=30, alpha=0.6, label=scenario, 
                    color=colors[j], density=True)
    ax_dist.set_title(f'{param.title()} Distribution Comparison')
    ax_dist.set_xlabel(param.title())
    ax_dist.set_ylabel('Density')
    ax_dist.legend()
    ax_dist.grid(True, alpha=0.3)
    
    # Box plot comparison
    ax_box = axes[i, 1]
    box_data = [data[param] for data in scenarios.values()]
    box_plot = ax_box.boxplot(box_data, labels=scenario_names, patch_artist=True)
    for patch, color in zip(box_plot['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    ax_box.set_title(f'{param.title()} Box Plot Comparison')
    ax_box.set_ylabel(param.title())
    ax_box.tick_params(axis='x', rotation=45)
    ax_box.grid(True, alpha=0.3)

# Create a comprehensive drift metrics comparison
ax_metrics = axes[2, :]
ax_metrics = plt.subplot(3, 1, 3)  # Span full width

# Calculate drift metrics for each scenario
drift_metrics_results = {}
reference_data = baseline_data[feature_cols]

for scenario_name, scenario_data in scenarios.items():
    if scenario_name == 'Baseline':
        continue
    
    current_data = scenario_data[feature_cols]
    drift_info = detect_drift(reference_data, current_data, DriftConfig())
    
    # Extract key metrics
    temp_psi = drift_info['drift_scores'].get('temperature_psi', 0)
    pressure_psi = drift_info['drift_scores'].get('pressure_psi', 0)
    temp_ks_p = drift_info['drift_scores'].get('temperature_ks_p', 1)
    pressure_ks_p = drift_info['drift_scores'].get('pressure_ks_p', 1)
    
    drift_metrics_results[scenario_name] = {
        'Temperature PSI': temp_psi,
        'Pressure PSI': pressure_psi,
        'Temperature KS p-value': temp_ks_p,
        'Pressure KS p-value': pressure_ks_p
    }

# Plot drift metrics
metrics_df = pd.DataFrame(drift_metrics_results).T
metrics_df[['Temperature PSI', 'Pressure PSI']].plot(kind='bar', ax=ax_metrics, 
                                                    color=['red', 'blue'], alpha=0.7)
ax_metrics.axhline(y=0.2, color='orange', linestyle='--', linewidth=2, label='PSI Warning Threshold')
ax_metrics.axhline(y=0.1, color='green', linestyle='--', linewidth=2, label='PSI Normal Threshold')
ax_metrics.set_title('Drift Detection Metrics (PSI Scores)')
ax_metrics.set_ylabel('PSI Score')
ax_metrics.set_xlabel('Drift Scenario')
ax_metrics.legend()
ax_metrics.grid(True, alpha=0.3)
ax_metrics.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("📊 Drift visualization complete")
print("\n📈 Drift Metrics Summary:")
print(metrics_df.round(4))

## 7. Comprehensive Drift Detection

Let's run our comprehensive drift detection system on each scenario and analyze the results.

In [ ]:
# Evaluate model performance under different drift conditions
print("🔍 Evaluating model performance under drift conditions...\n")

drift_evaluation_results = {}

for scenario_name, scenario_data in scenarios.items():
    if scenario_name == 'Baseline':
        continue
    
    print(f"📊 Evaluating {scenario_name}...")
    
    # Prepare data
    X_drift = scenario_data[feature_cols]
    y_drift = scenario_data['target'].values
    
    # Use a subset for faster evaluation
    n_samples = min(400, len(X_drift))
    X_drift_subset = X_drift.iloc[:n_samples]
    y_drift_subset = y_drift[:n_samples]
    
    # Evaluate with drift detection
    eval_results = pipeline.evaluate(X_drift_subset, y_drift_subset)
    drift_evaluation_results[scenario_name] = eval_results
    
    # Print key findings
    print(f"   📈 MAE: {eval_results['mae']:.3f} (baseline: {baseline_metrics['mae']:.3f})")
    print(f"   📈 PWS: {eval_results['pws_percent']:.1f}% (baseline: {baseline_metrics['pws_percent']:.1f}%)")
    print(f"   🚨 Drift detected: {any(eval_results['alert_flags'].values())}")
    print(f"   📊 Active alerts: {sum(eval_results['alert_flags'].values())}")
    print()

print("✅ Drift evaluation complete")

## 8. Alert Dashboard Simulation

Let's create a dashboard-style visualization showing how our alert system would look in production.

In [ ]:
# Create alert dashboard visualization
fig = plt.figure(figsize=(16, 12))
fig.suptitle('Semiconductor Model Monitoring Dashboard', fontsize=18, fontweight='bold')

# Create grid layout
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Overall Health Score
ax_health = fig.add_subplot(gs[0, 0])
health_scores = []
scenario_labels = []

for scenario_name, results in drift_evaluation_results.items():
    # Calculate health score (0-100)
    performance_score = max(0, 100 - (results['mae'] / baseline_metrics['mae'] - 1) * 100)
    drift_penalty = sum(results['alert_flags'].values()) * 10
    health_score = max(0, performance_score - drift_penalty)
    
    health_scores.append(health_score)
    scenario_labels.append(scenario_name)

colors_health = ['green' if score > 80 else 'orange' if score > 50 else 'red' for score in health_scores]
bars = ax_health.bar(scenario_labels, health_scores, color=colors_health, alpha=0.7)
ax_health.set_title('Model Health Score', fontweight='bold')
ax_health.set_ylabel('Health Score (0-100)')
ax_health.set_ylim(0, 100)
ax_health.tick_params(axis='x', rotation=45)

# Add score labels
for bar, score in zip(bars, health_scores):
    height = bar.get_height()
    ax_health.text(bar.get_x() + bar.get_width()/2., height + 2,
                   f'{score:.0f}', ha='center', va='bottom', fontweight='bold')

# 2. Performance Degradation
ax_perf = fig.add_subplot(gs[0, 1])
mae_values = [results['mae'] for results in drift_evaluation_results.values()]
baseline_mae_line = [baseline_metrics['mae']] * len(mae_values)

ax_perf.plot(scenario_labels, mae_values, 'o-', color='red', linewidth=2, markersize=8, label='Current MAE')
ax_perf.plot(scenario_labels, baseline_mae_line, '--', color='blue', linewidth=2, label='Baseline MAE')
ax_perf.fill_between(scenario_labels, baseline_mae_line, 
                     [x * 1.15 for x in baseline_mae_line], alpha=0.2, color='orange', label='Warning Zone')
ax_perf.set_title('Performance Trend', fontweight='bold')
ax_perf.set_ylabel('Mean Absolute Error')
ax_perf.legend(loc='upper left')
ax_perf.tick_params(axis='x', rotation=45)
ax_perf.grid(True, alpha=0.3)

# 3. Alert Summary
ax_alerts = fig.add_subplot(gs[0, 2])
alert_counts = [sum(results['alert_flags'].values()) for results in drift_evaluation_results.values()]
colors_alerts = ['green' if count == 0 else 'orange' if count < 5 else 'red' for count in alert_counts]

bars = ax_alerts.bar(scenario_labels, alert_counts, color=colors_alerts, alpha=0.7)
ax_alerts.set_title('Active Alerts', fontweight='bold')
ax_alerts.set_ylabel('Number of Alerts')
ax_alerts.tick_params(axis='x', rotation=45)

# 4. Drift Heatmap
ax_heatmap = fig.add_subplot(gs[1, :])
drift_matrix = []
drift_features = ['temperature_psi', 'pressure_psi', 'flow_psi', 'time_psi']

for scenario_name, results in drift_evaluation_results.items():
    drift_row = [results['drift_scores'].get(feature, 0) for feature in drift_features]
    drift_matrix.append(drift_row)

im = ax_heatmap.imshow(drift_matrix, cmap='RdYlGn_r', aspect='auto', vmin=0, vmax=0.5)
ax_heatmap.set_xticks(range(len(drift_features)))
ax_heatmap.set_xticklabels([f.replace('_psi', '').title() for f in drift_features])
ax_heatmap.set_yticks(range(len(scenario_labels)))
ax_heatmap.set_yticklabels(scenario_labels)
ax_heatmap.set_title('Drift Detection Heatmap (PSI Scores)', fontweight='bold')

# Add colorbar
cbar = plt.colorbar(im, ax=ax_heatmap, orientation='horizontal', pad=0.1, shrink=0.8)
cbar.set_label('PSI Score')

# Add text annotations
for i in range(len(scenario_labels)):
    for j in range(len(drift_features)):
        text = ax_heatmap.text(j, i, f'{drift_matrix[i][j]:.3f}',
                              ha="center", va="center", color="black", fontweight='bold')

# 5. Manufacturing Metrics
ax_mfg = fig.add_subplot(gs[2, :])
pws_values = [results['pws_percent'] for results in drift_evaluation_results.values()]
loss_values = [results['estimated_loss']/1000 for results in drift_evaluation_results.values()]

ax_mfg_twin = ax_mfg.twinx()

line1 = ax_mfg.plot(scenario_labels, pws_values, 'o-', color='green', linewidth=3, 
                    markersize=10, label='PWS %')
line2 = ax_mfg_twin.plot(scenario_labels, loss_values, 's-', color='red', linewidth=3, 
                         markersize=10, label='Est. Loss ($K)')

ax_mfg.set_xlabel('Drift Scenario')
ax_mfg.set_ylabel('Prediction Within Spec (%)', color='green')
ax_mfg_twin.set_ylabel('Estimated Loss ($K)', color='red')
ax_mfg.set_title('Manufacturing Impact Metrics', fontweight='bold')
ax_mfg.tick_params(axis='x', rotation=45)
ax_mfg.grid(True, alpha=0.3)

# Add combined legend
lines1, labels1 = ax_mfg.get_legend_handles_labels()
lines2, labels2 = ax_mfg_twin.get_legend_handles_labels()
ax_mfg.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.tight_layout()
plt.show()

print("📊 Alert dashboard visualization complete")

## 9. Production Deployment Simulation

Let's simulate how this monitoring system would work in a production environment with streaming data.

In [ ]:
# Simulate production monitoring with streaming data
print("🏭 Simulating production monitoring...")

# Create time series data simulating a production day
import datetime

# Generate hourly data for 24 hours
time_points = pd.date_range(start='2024-01-01 00:00:00', periods=24, freq='1H')
monitoring_log = []

for i, timestamp in enumerate(time_points):
    # Simulate gradual drift throughout the day
    drift_factor = i / 24.0 * 0.3  # Gradual drift over 24 hours
    
    # Generate data with increasing drift
    hourly_data = generate_yield_process(n=50, seed=42 + i)
    if drift_factor > 0:
        hourly_data = generate_drift_data(hourly_data, drift_strength=drift_factor)
    
    # Evaluate model performance
    X_hourly = hourly_data[feature_cols]
    y_hourly = hourly_data['target'].values
    
    eval_results = pipeline.evaluate(X_hourly, y_hourly)
    
    # Log results
    log_entry = {
        'timestamp': timestamp,
        'hour': i,
        'mae': eval_results['mae'],
        'rmse': eval_results['rmse'],
        'pws_percent': eval_results['pws_percent'],
        'estimated_loss': eval_results['estimated_loss'],
        'drift_detected': any(eval_results['alert_flags'].values()),
        'alert_count': sum(eval_results['alert_flags'].values()),
        'temp_psi': eval_results['drift_scores'].get('temperature_psi', 0),
        'pressure_psi': eval_results['drift_scores'].get('pressure_psi', 0)
    }
    monitoring_log.append(log_entry)

# Convert to DataFrame
monitoring_df = pd.DataFrame(monitoring_log)
monitoring_df.set_index('timestamp', inplace=True)

print(f"✅ Generated {len(monitoring_df)} hours of monitoring data")
print(f"🚨 Drift detected in {monitoring_df['drift_detected'].sum()} hours")
print(f"📊 Average alerts per hour: {monitoring_df['alert_count'].mean():.1f}")

# Display sample of monitoring log
print("\n📋 Sample Monitoring Log:")
print(monitoring_df[['hour', 'mae', 'pws_percent', 'drift_detected', 'alert_count']].head(10))

In [ ]:
# Visualize production monitoring timeline
fig, axes = plt.subplots(4, 1, figsize=(15, 16))
fig.suptitle('24-Hour Production Monitoring Timeline', fontsize=16, fontweight='bold')

# 1. Performance metrics over time
ax1 = axes[0]
ax1_twin = ax1.twinx()

line1 = ax1.plot(monitoring_df.index, monitoring_df['mae'], 'b-', linewidth=2, label='MAE')
line2 = ax1_twin.plot(monitoring_df.index, monitoring_df['pws_percent'], 'g-', linewidth=2, label='PWS %')

# Add baseline reference
ax1.axhline(y=baseline_metrics['mae'], color='blue', linestyle='--', alpha=0.5, label='Baseline MAE')
ax1_twin.axhline(y=baseline_metrics['pws_percent'], color='green', linestyle='--', alpha=0.5, label='Baseline PWS')

ax1.set_ylabel('Mean Absolute Error', color='blue')
ax1_twin.set_ylabel('Prediction Within Spec (%)', color='green')
ax1.set_title('Model Performance Over Time')
ax1.grid(True, alpha=0.3)

# Combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twin.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# 2. Drift scores over time
ax2 = axes[1]
ax2.plot(monitoring_df.index, monitoring_df['temp_psi'], 'r-', linewidth=2, label='Temperature PSI')
ax2.plot(monitoring_df.index, monitoring_df['pressure_psi'], 'b-', linewidth=2, label='Pressure PSI')
ax2.axhline(y=0.2, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Warning Threshold')
ax2.axhline(y=0.1, color='green', linestyle='--', linewidth=2, alpha=0.7, label='Normal Threshold')
ax2.set_ylabel('PSI Score')
ax2.set_title('Drift Detection Scores')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Alert timeline
ax3 = axes[2]
alert_colors = ['red' if count > 5 else 'orange' if count > 0 else 'green' 
                for count in monitoring_df['alert_count']]
bars = ax3.bar(monitoring_df.index, monitoring_df['alert_count'], 
               color=alert_colors, alpha=0.7, width=0.8/24)
ax3.set_ylabel('Number of Alerts')
ax3.set_title('Alert Activity Timeline')
ax3.grid(True, alpha=0.3)

# 4. Business impact
ax4 = axes[3]
cumulative_loss = monitoring_df['estimated_loss'].cumsum() / 1000  # Convert to thousands
ax4.plot(monitoring_df.index, cumulative_loss, 'r-', linewidth=3, label='Cumulative Loss')
ax4.fill_between(monitoring_df.index, 0, cumulative_loss, alpha=0.3, color='red')
ax4.set_ylabel('Cumulative Loss ($K)')
ax4.set_xlabel('Time')
ax4.set_title('Business Impact Over Time')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Format x-axis for all subplots
for ax in axes:
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("📊 Production monitoring timeline visualization complete")
print(f"💰 Total estimated loss over 24 hours: ${cumulative_loss.iloc[-1]:,.0f}K")
print(f"📈 Performance degradation: {((monitoring_df['mae'].iloc[-1] / baseline_metrics['mae']) - 1) * 100:.1f}%")

## 10. Key Insights and Recommendations

Let's summarize the key insights from our monitoring analysis.

In [ ]:
# Generate comprehensive monitoring summary
print("📊 MONITORING & MAINTENANCE ANALYSIS SUMMARY")
print("=" * 60)

print("\n🎯 BASELINE MODEL PERFORMANCE:")
print(f"   • MAE: {baseline_metrics['mae']:.3f}")
print(f"   • RMSE: {baseline_metrics['rmse']:.3f}")
print(f"   • R²: {baseline_metrics['r2']:.3f}")
print(f"   • PWS: {baseline_metrics['pws_percent']:.1f}%")
print(f"   • Estimated Loss: ${baseline_metrics['estimated_loss']:,.0f}")

print("\n🚨 DRIFT DETECTION RESULTS:")
for scenario_name, results in drift_evaluation_results.items():
    performance_change = ((results['mae'] / baseline_metrics['mae']) - 1) * 100
    pws_change = results['pws_percent'] - baseline_metrics['pws_percent']
    
    print(f"\n   📉 {scenario_name.upper()}:")
    print(f"      • Performance change: {performance_change:+.1f}%")
    print(f"      • PWS change: {pws_change:+.1f}%")
    print(f"      • Drift alerts: {sum(results['alert_flags'].values())}")
    print(f"      • Max PSI score: {max(v for k, v in results['drift_scores'].items() if 'psi' in k):.3f}")

print("\n⚡ PRODUCTION SIMULATION INSIGHTS:")
print(f"   • Hours with drift detected: {monitoring_df['drift_detected'].sum()}/24")
print(f"   • Peak alert count: {monitoring_df['alert_count'].max()}")
print(f"   • Performance degradation at end: {((monitoring_df['mae'].iloc[-1] / baseline_metrics['mae']) - 1) * 100:.1f}%")
print(f"   • Total business impact: ${cumulative_loss.iloc[-1]:,.0f}K")

print("\n🔧 RECOMMENDED ACTIONS:")
print("   1. 🎯 Set PSI thresholds:")
print("      • Normal: < 0.1 (green)")
print("      • Warning: 0.1-0.2 (yellow)")
print("      • Critical: > 0.2 (red)")

print("\n   2. 📊 Monitor key metrics:")
print("      • Model accuracy (MAE/RMSE)")
print("      • Prediction Within Spec (PWS)")
print("      • Business impact (Estimated Loss)")
print("      • Drift scores (PSI, KS-test, Wasserstein)")

print("\n   3. 🚨 Alert escalation levels:")
print("      • Info: PSI 0.1-0.15, performance degradation < 10%")
print("      • Warning: PSI 0.15-0.25, performance degradation 10-20%")
print("      • Critical: PSI > 0.25, performance degradation > 20%")

print("\n   4. 🔄 Maintenance schedule:")
print("      • Daily: Review alert logs and performance trends")
print("      • Weekly: Analyze drift patterns and update thresholds")
print("      • Monthly: Model retraining evaluation")
print("      • Quarterly: Comprehensive performance review")

print("\n   5. 🏭 Production deployment:")
print("      • Use MLflow for experiment tracking")
print("      • Implement blue-green deployment")
print("      • Set up automated rollback triggers")
print("      • Monitor business KPIs continuously")

print("\n" + "=" * 60)
print("✅ ANALYSIS COMPLETE - READY FOR PRODUCTION DEPLOYMENT")
print("=" * 60)

## 11. Save Model and Monitoring Configuration

Finally, let's save our trained model and monitoring configuration for production use.

In [ ]:
# Save model and monitoring configuration
from pathlib import Path
import json

# Create output directory
output_dir = Path('monitoring_output')
output_dir.mkdir(exist_ok=True)

# Save the trained model
model_path = output_dir / 'production_model.joblib'
pipeline.save(model_path)
print(f"💾 Model saved to: {model_path}")

# Save monitoring configuration
monitoring_config = {
    'drift_thresholds': {
        'psi_warning': 0.1,
        'psi_critical': 0.2,
        'ks_p_warning': 0.05,
        'ks_p_critical': 0.01,
        'wasserstein_warning': 0.5,
        'wasserstein_critical': 1.0
    },
    'performance_thresholds': {
        'mae_warning_pct': 10,
        'mae_critical_pct': 20,
        'pws_warning_min': 90,
        'pws_critical_min': 80
    },
    'alert_settings': {
        'consecutive_violations': 2,
        'evaluation_frequency_minutes': 15,
        'reporting_frequency_hours': 1
    },
    'baseline_metrics': {
        'mae': float(baseline_metrics['mae']),
        'rmse': float(baseline_metrics['rmse']),
        'r2': float(baseline_metrics['r2']),
        'pws_percent': float(baseline_metrics['pws_percent']),
        'estimated_loss': float(baseline_metrics['estimated_loss'])
    }
}

config_path = output_dir / 'monitoring_config.json'
with open(config_path, 'w') as f:
    json.dump(monitoring_config, f, indent=2)
print(f"⚙️  Monitoring config saved to: {config_path}")

# Save monitoring results
results_path = output_dir / 'monitoring_results.csv'
monitoring_df.to_csv(results_path)
print(f"📊 Monitoring results saved to: {results_path}")

# Create deployment script
deployment_script = '''#!/bin/bash
# Production deployment script for semiconductor monitoring

echo "🚀 Deploying Model Monitoring System..."

# Start MLflow server
mlflow server --backend-store-uri sqlite:///production_mlflow.db \
              --default-artifact-root ./mlruns \
              --port 5000 &

echo "📊 MLflow server started on port 5000"

# Set environment variables
export MLFLOW_TRACKING_URI=http://localhost:5000
export MODEL_PATH=monitoring_output/production_model.joblib
export CONFIG_PATH=monitoring_output/monitoring_config.json

echo "✅ Environment configured"
echo "🔗 MLflow UI: http://localhost:5000"
echo "📋 Ready for production monitoring!"
'''

deploy_script_path = output_dir / 'deploy.sh'
with open(deploy_script_path, 'w') as f:
    f.write(deployment_script)
deploy_script_path.chmod(0o755)  # Make executable
print(f"🚀 Deployment script saved to: {deploy_script_path}")

print("\n✅ All artifacts saved successfully!")
print(f"📁 Output directory: {output_dir.absolute()}")
print("\n🔧 To deploy in production:")
print(f"   cd {output_dir}")
print("   ./deploy.sh")

## 🎯 Summary

Congratulations! You've successfully completed the monitoring and maintenance module. Here's what you've learned:

### 🔬 **Technical Skills Gained**
- Set up MLflow for experiment tracking and model registry
- Implemented multiple drift detection methods (PSI, KS-test, Wasserstein)
- Created manufacturing-specific metrics (PWS, Estimated Loss)
- Built comprehensive alerting and monitoring dashboards
- Simulated production deployment scenarios

### 📊 **Key Insights**
- Different drift types require different detection strategies
- Manufacturing metrics provide business-relevant model evaluation
- Alert thresholds must balance sensitivity vs. false alarm rates
- Continuous monitoring is essential for production ML systems
- Proper tooling (MLflow) significantly improves model lifecycle management

### 🚀 **Next Steps**
1. **Deploy** the monitoring system in your environment
2. **Customize** thresholds based on your specific process requirements
3. **Integrate** with existing manufacturing systems and dashboards
4. **Train** your team on interpreting alerts and taking corrective actions
5. **Establish** maintenance schedules and escalation procedures

### 💡 **Best Practices Learned**
- Monitor both statistical drift and business impact
- Use multiple drift detection methods for robustness
- Implement gradual alerting (info → warning → critical)
- Track model lineage and approval workflows
- Plan for graceful degradation and fallback strategies

---

**🎉 You're now ready to implement robust model monitoring in production semiconductor manufacturing environments!**